<a href="https://colab.research.google.com/github/phsieg/gan_artwork_open/blob/main/tf_gan_tutorial_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook is for testing the tutorial at https://www.tensorflow.org/tutorials/generative/cyclegan

# Find Local Modules Help

In [ ]:
help('modules')

# Testing Pad

In [33]:
band = read_multiple_binary_files('test_folder')
band = read_multiple_binary_files('test_folder/Untitled Folder')


test_folder/Untitled Folder is not a known image file, i.e. JPG, IMG or PNG
test_folder/Portrait_duo.jpg has been read succesfully!
test_folder/Portrait_duo_artifact_triangles.jpg.png has been read succesfully!
test_folder/Untitled Folder/anscombe.json is not a known image file, i.e. JPG, IMG or PNG


# File handling

In [32]:
from posixpath import splitext
from pathlib import Path

### Image files

knownImageFileTypes = ['.jpg', '.jpeg', '.JPG', '.JPEG', '.png', '.PNG', \
'.img', '.IMG']

def read_multiple_binary_files(path):
  p = Path(path)
  list_of_files = []
  for child in p.iterdir():
    try:
      if is_Image_File(child):
        file_binaries = read_single_binary_file(child)
        list_of_files.append(file_binaries)
        print(str(child) + " has been read succesfully!")
      else:
        print(str(child) + " is not a known image file, i.e. JPG, IMG or PNG")
    except IsADirectoryError as N:
      print(str(child) + " is not a file! Error caught: " + str(N))
  return list_of_files
      
def is_Image_File(path):
  isImage = False
  if splitext(path)[1] in knownImageFileTypes:
    isImage = True
  return isImage

def read_single_binary_file(path):
  with open(path, 'rb') as f:
    return f.read()
  
def write_single_binary_file(path):
  with open(path, 'wb') as f:
    f.write()

### Text files

# tbd

# Tensorflow

In [ ]:
import tensorflow as tf
# for drawing generated images by plotting
import matplotlib.pyplot as plt 

# Program Execution


In [ ]:
islands = read_multiple_binary_files('.data/input/islands/')
bandfotos = read_multiple_binary_files('.data/input/bandsfotos/')

# do ML stuff here with islands and bandfotos...

write_single_binary_file('.data/output/artwork.png')